# Description of this Notebook

This notebook is designed to process and analyze data from the French government website:  
[Emissions de CO2 et de polluants des véhicules commercialisés en France](https://www.data.gouv.fr/fr/datasets/emissions-de-co2-et-de-polluants-des-vehicules-commercialises-en-france/#_).  

The workflow includes downloading Excel files, converting them to CSV format, and appending the year to each filename (e.g., `_year.csv`). These processed files are stored in the `project/data` directory and excluded from version control via `.gitignore`. The notebook prepares a unified DataFrame covering car registration data in France from 2001 to 2014. The dataset includes technical specifications and CO2 emissions.

## Important Note
To run this script successfully, the data files must be present in the `data` directory of the repository. If stored elsewhere, update the data path in the script accordingly. Ensure all CSV files are accessible in the defined `data_path`.

---

## Processing Workflow

### 1. Data Loading
- Remove empty rows and columns.
- Eliminate unnecessary indexes.
- Collect column names into a `column_names` list.
- Append year information to each DataFrame.

### 2. Data Preprocessing
- Replace commas with decimal points for numeric values.
- Remove duplicate columns containing redundant information.

### 3. Data Combination
- Combine all DataFrames into a single list for batch processing.

### 4. Data Overview
- Analyze the structure of the data, including shapes and column names.

### 5. Data Standardization
- Map and relabel column names into standardized groups.
- Assign clear and consistent English labels.

### 6. Data Integration
- Rename any remaining duplicate columns.
- Merge all DataFrames into a unified DataFrame (`data_all`).

### 7. Data Type Conversion
- Convert object-type columns to floats where applicable.
- Fill missing entries with NaN values.

### 8. Data Export
- Save the final unified DataFrame as `data_all_french.csv`.



In [4]:
# Import required libraries
# ----------------------

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization 
import matplotlib.pyplot as plt  # Matplotlib for basic plotting
import seaborn as sns           # Seaborn for statistical visualizations
import plotly.express as px     # Plotly Express for interactive plots
#%matplotlib inline

# System utilities
import warnings
import os

# Suppress warnings due to font compatibility issues
warnings.filterwarnings("ignore")


In [6]:
# Retrieving data 

# Path to the neighbouring data folder
data_path = os.path.join(os.path.dirname(os.getcwd()), 'data')

# Reading all data and save them in global variables 
# Iterate through all files in the Data directory
for filename in os.listdir(data_path):
    # Check if the file is a .csv file
    if filename.endswith('.csv'):
        # Create the full path to the file
        file_path = os.path.join(data_path, filename)
        
        # Extract the last 4 digits of the filename (before the extension)
        last_four_digits = filename[-8:-4]  # Assumption: There are always 4 digits before the file extension
        
        # Make the year variable 
        year = int(last_four_digits)

        # Read the .csv file into a DataFrame
        df = pd.read_csv(file_path, encoding='latin1', sep=';', index_col=None)

        ### Deleting empty columns and rows for each df before saving it in a global variable 
        # Identify columns that contain only NaN values
        nan_columns = df.columns[df.isna().all()].tolist()    
        # Drop the columns with only NaN values
        df.drop(columns=nan_columns, inplace=True)
        # Delete all rows that are completely empty (i.e., contain only NaN values)
        empty_rows = df[df.isna().all(axis=1)].index
        df.drop(index=empty_rows, inplace=True)

        # Deleting indexing for later better working
        df.reset_index(drop=True)

        # Looking at the columns and the shape 
        column_names = df.columns.tolist()
        #print("Column names in df of year:", year, "\n List:", column_names)
        print(df.shape)
        # Add an additional year column to each df
        df['YEAR'] = year

        # Save the DataFrame under the desired name
        globals()[f'df_french_{last_four_digits}'] = df


(40052, 25)
(19477, 15)
(37816, 16)


ValueError: invalid literal for int() with base 10: 'ench'

In [63]:
# Working specifically with dfs analyse content 

# Settings output of terminal to max to be able to work with all the information 
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

# Special treatment for some extra dfs
# Replacing commas with periods in entries 
df_french_2011 = df_french_2011.replace(',', '.', regex=True)
df_french_2012 = df_french_2012.replace(',', '.', regex=True)

# Deleting double columns which contain redundant data 
df_french_2002 = df_french_2002.drop(df_french_2002.columns[4], axis=1)
df_french_2009 = df_french_2009.drop(df_french_2009.columns[13], axis=1)
df_french_2010 = df_french_2010.drop(df_french_2010.columns[13], axis=1)
df_french_2011 = df_french_2011.drop(df_french_2011.columns[1], axis=1)
df_french_2011 = df_french_2011.drop(df_french_2011.columns[3], axis=1)
df_french_2011 = df_french_2011.drop(df_french_2011.columns[2], axis=1)

"""
print(df_french_2001.head(3))
print(df_french_2002.head(3))
print(df_french_2003.head(3))
print(df_french_2004.head(3))
print(df_french_2005.head(3))
print(df_french_2006.head(3))
print(df_french_2007.head(3))
print(df_french_2008.head(3))
print(df_french_2009.head(3))
print(df_french_2010.head(3))
print(df_french_2011.head(3))
print(df_french_2012.head(3))
"""


'\nprint(df_french_2001.head(3))\nprint(df_french_2002.head(3))\nprint(df_french_2003.head(3))\nprint(df_french_2004.head(3))\nprint(df_french_2005.head(3))\nprint(df_french_2006.head(3))\nprint(df_french_2007.head(3))\nprint(df_french_2008.head(3))\nprint(df_french_2009.head(3))\nprint(df_french_2010.head(3))\nprint(df_french_2011.head(3))\nprint(df_french_2012.head(3))\n'

In [64]:
# Combine all df to one list to work simultaneous one one list of dfs 
dataframes = [
    df_french_2001,
    df_french_2002,
    df_french_2003,
    df_french_2004,
    df_french_2005,
    df_french_2006,
    df_french_2007,
    df_french_2008,
    df_french_2009,
    df_french_2010,
    df_french_2011,
    df_french_2012
]

# Corresponding names for the DataFrames
df_names = [
    'df_french_2001',
    'df_french_2002',
    'df_french_2003',
    'df_french_2004',
    'df_french_2005',
    'df_french_2006',
    'df_french_2007',
    'df_french_2008',
    'df_french_2009',
    'df_french_2010',
    'df_french_2011',
    'df_french_2012'
]

# Iterate through each DataFrame and print its column names
for df, name in zip(dataframes, df_names):
    print(f"{name}, column names: {list(df.columns)}\n")

df_french_2001, column names: ['MARQUES', 'MODELE', 'TYP. MINES', 'CNIT', 'CARB', 'CV', 'PUISS.', 'BV', 'Urbain', 'Ex.Urb', 'Mixte', 'CO2', 'YEAR']

df_french_2002, column names: ['MARQUE', 'MODELE VERSION', 'CNIT', 'Unnamed: 4', 'MINE', 'ENERGIE', 'puissance fiscale', 'puissance reelle', 'bv', 'urb', 'ex-urb', 'mixte', 'CO2', 'YEAR']

df_french_2003, column names: ['MARQUE', 'MODELE VERSION', 'CNIT', 'MINE', 'ENERGIE', 'puissance fiscale', 'puissance reelle', 'bv', 'urb', 'ex-urb', 'mixte', 'CO2', 'IMMAT', 'YEAR']

df_french_2004, column names: ['Marques', 'ModÃ¨les, Versions', 'CNIT', 'CARBURANT', 'Puiss Administrative', 'Puissance max', 'BV', 'Conso urb', 'Conso ex-urb', 'Conso mixte', 'CO2', 'Unnamed: 11', 'YEAR']

df_french_2005, column names: ['MARQUE', 'MODELE VERSION', 'CNIT', 'puissance fiscale', 'puissance reelle', 'bv', 'urb', 'ex-urb', 'mixte', 'CO2', 'ENERGIE', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'YEAR']

df_french_2006, column names: ['MARQUES', 'MODELE', 'TYP. M

In [65]:
# Getting an overview over all the df 

# Print out the shape and column names of all DataFrames at the end
print("Final shapes and column names of all DataFrames:")
for i, df in enumerate(dataframes):
    print(f"DataFrame {i + 1}:")
    print(f"Shape: {df.shape}")
    print(f"Column names: {df.columns.tolist()}\n")

# We should be able to make 12 overlay columns 

Final shapes and column names of all DataFrames:
DataFrame 1:
Shape: (5919, 13)
Column names: ['MARQUES', 'MODELE', 'TYP. MINES', 'CNIT', 'CARB', 'CV', 'PUISS.', 'BV', 'Urbain', 'Ex.Urb', 'Mixte', 'CO2', 'YEAR']

DataFrame 2:
Shape: (5183, 14)
Column names: ['MARQUE', 'MODELE VERSION', 'CNIT', 'Unnamed: 4', 'MINE', 'ENERGIE', 'puissance fiscale', 'puissance reelle', 'bv', 'urb', 'ex-urb', 'mixte', 'CO2', 'YEAR']

DataFrame 3:
Shape: (5398, 14)
Column names: ['MARQUE', 'MODELE VERSION', 'CNIT', 'MINE', 'ENERGIE', 'puissance fiscale', 'puissance reelle', 'bv', 'urb', 'ex-urb', 'mixte', 'CO2', 'IMMAT', 'YEAR']

DataFrame 4:
Shape: (138, 13)
Column names: ['Marques', 'ModÃ¨les, Versions', 'CNIT', 'CARBURANT', 'Puiss Administrative', 'Puissance max', 'BV', 'Conso urb', 'Conso ex-urb', 'Conso mixte', 'CO2', 'Unnamed: 11', 'YEAR']

DataFrame 5:
Shape: (7980, 15)
Column names: ['MARQUE', 'MODELE VERSION', 'CNIT', 'puissance fiscale', 'puissance reelle', 'bv', 'urb', 'ex-urb', 'mixte', 'CO2', '

In [66]:
# Here the relabeling of different column names to one will happen 
# The relabeling from french to a commong english collum name 
# Additional information on what the column names mean 

# Mapping

column_mapping = {
    # The manufacturer or brand of the vehicle (e.g., Opel, Ford).
    'Brand': ['MARQUE', 'MARQUES', 'lib_mrq', 'Marques'],  
    # The specific model or version of the vehicle (e.g., Corsa, Fiesta).
    'Model': ['MODELE VERSION', 'MODELE_VERSION', 'MODELE', 'Modèles, Versions', 'lib_mod', 'ModÃ¨les, Versions', 'lib_mod_doss'],  
    # Vehicle's technical type approval or classification. (e.g., MJD1102J)
    'Type Mines': ['TYP. MINES', 'TYPE 2', 'TYPE', 'MINE', 'tvv'],
    # The National Type Identification Code, unique to a vehicle in some countries. (e.g., M10ALFVP000M091)
    'CNIT': ['CNIT', 'cnit'],
    # The type of fuel the vehicle uses (e.g., ES, EL, GO)
    'Fuel Type': ['ENERGIE', 'carburant',  'typ_cbr', 'CARBURANT', 'CARB'],  
    # The actual power output of the engine, measured in horsepower or kilowatts. (e.g., 45.0, 77.0)
    'Engine Power': ['PUISSANCE', 'puissance reelle',  'puiss_max', 'PUISS.'],  
    # Fiscal horsepower, often used to calculate taxes on vehicles. (e.g., 4, 7)
    'Fiscal Power': ['Puiss Administrative', 'puissance fiscale'],
    # The type of transmission (e.g., M 5, V 0, A 6). 
    # A, M and V stand for different transmissions 
    'Transmission': ['bv', 'BV', 'typ_boite_nb_rapp'], 
    # Fuel consumption in urban or city driving conditions. (e.g., 11.3, 6.4)
    'Urban Consumption': ['urb', 'Urbain', 'conso_urb', 'Conso urb', 'URBAIN'], 
    # Fuel consumption in extra-urban (highway or countryside) driving conditions.(e.g., 11.3, 6.4)
    'Extra Urban Consumption': ['ex-urb', 'Ex.Urb', 'conso_exurb', 'Conso ex-urb', 'EX_URBAIN'],  
    # Combined or average fuel consumption across different driving conditions. (e.g., 11.3, 6.4)
    'Combined Consumption': ['mixte', 'Mixte', 'conso_mixte', 'Conso mixte', 'MIXTE'], 
    #The amount of CO2 the vehicle emits, usually measured in grams per kilometer (g/km).(e.g., 136.0, 196)
    'CO2 Emissions': ['CO2', 'co2'], 
    # The admission year of the car. (e.g., 2001, 2012)
    'YEAR': ['YEAR'],  
    # Energy efficiency label, indicating the environmental rating of the vehicle. (e.g., A, B, C, E) 
    'Energy Label': ['ENERGIE', 'ETIQUETTE ENERGIE'], 
    # A government incentive (bonus) or penalty (malus) based on the vehicle's emissions. (e.g., -5000) 
    'Bonus/Malus': ['BONUS(-)/MALUS( )', 'BONUS(-)/MALUS(+)'],
    # Likely refers to a technical or legal classification in the vehicle's registration data. (e.g., 70/220*2003/76EURO4)
    'V9 Field': ['champ_v9', 'CHAMP_V9'],
    # Body type of the car e.g., BERLINE
    'Body Type': ['Carrosserie', 'CARROSSERIE'],  
    # The vehicle's registration date. (e.g. Mar-03)
    'Registration': ['IMMAT', 'date_maj', 'DATE_MAJ'],

    # Unnamed columns 
    'Unnamed': ['Unnamed: 4', 'Unnamed: 5', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17'],
}


# Create a dictionary for column name mapping
flattened_dict = {}

# Flatten the column mapping into a single dictionary for renaming
for key, values in column_mapping.items():
    for value in values:
        flattened_dict[value] = key

# Print the flattened dictionary to check the final column mappings
print(flattened_dict)

# Iterate through all DataFrames and rename columns according to the mapping
for df in dataframes:
    # Rename the columns based on the flattened dictionary
    df.rename(columns=flattened_dict, inplace=True)

# Optional: Print the new column names for each DataFrame
for i, df in enumerate(dataframes):
    print(f"New column names for DataFrame {i + 1}: {df.columns}\n")
    print(df.shape)


{'MARQUE': 'Brand', 'MARQUES': 'Brand', 'lib_mrq': 'Brand', 'Marques': 'Brand', 'MODELE VERSION': 'Model', 'MODELE_VERSION': 'Model', 'MODELE': 'Model', 'Modèles, Versions': 'Model', 'lib_mod': 'Model', 'ModÃ¨les, Versions': 'Model', 'lib_mod_doss': 'Model', 'TYP. MINES': 'Type Mines', 'TYPE 2': 'Type Mines', 'TYPE': 'Type Mines', 'MINE': 'Type Mines', 'tvv': 'Type Mines', 'CNIT': 'CNIT', 'cnit': 'CNIT', 'ENERGIE': 'Energy Label', 'carburant': 'Fuel Type', 'typ_cbr': 'Fuel Type', 'CARBURANT': 'Fuel Type', 'CARB': 'Fuel Type', 'PUISSANCE': 'Engine Power', 'puissance reelle': 'Engine Power', 'puiss_max': 'Engine Power', 'PUISS.': 'Engine Power', 'Puiss Administrative': 'Fiscal Power', 'puissance fiscale': 'Fiscal Power', 'bv': 'Transmission', 'BV': 'Transmission', 'typ_boite_nb_rapp': 'Transmission', 'urb': 'Urban Consumption', 'Urbain': 'Urban Consumption', 'conso_urb': 'Urban Consumption', 'Conso urb': 'Urban Consumption', 'URBAIN': 'Urban Consumption', 'ex-urb': 'Extra Urban Consumpti

In [67]:
# The combination of all data into one data_all data frame 

# Delete duplicate columns 
def deduplicate_columns(df):
    """Deduplicate DataFrame columns by appending a suffix."""
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [f"{dup}.{i+1}" if i > 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df

# Deduplicate column names
dataframes = [deduplicate_columns(df) for df in dataframes]

data_all = pd.concat(dataframes, axis=0, join='outer')

          Brand                 Model    Type Mines                          CNIT Fuel Type CV Engine Power Transmission Urban Consumption Extra Urban Consumption Combined Consumption CO2 Emissions  YEAR Unnamed Energy Label Fiscal Power Registration  Puissance max Unnamed.2 Unnamed.3 CINT Bonus/Malus  puiss_admin_98 V9 Field Model.2 dscom hybride co_typ_1   hc  nox hcnox ptcl  masse_ordma_min  masse_ordma_max Body Type gamme
0    VOLKSWAGEN           LUPO 3L TDI  MVW70C1R4385      6ESCANYX01AGFD5850021N0H        GO  3         45.0          A 5               3.6                     2.7                  3.0            81  2001     NaN          NaN          NaN          NaN            NaN       NaN       NaN  NaN         NaN             NaN      NaN     NaN   NaN     NaN      NaN  NaN  NaN   NaN  NaN              NaN              NaN       NaN   NaN
1    VOLKSWAGEN           LUPO 3L TDI  MVW70C1RX572      6ESCAYZX01AGFD5850021N0H        GO  3         45.0          A 5               3.6  

In [68]:
# Convert object type columns to flaots if possible 

# Information on total df 
print(data_all.shape)
print(data_all.info())

# Assuming 'data_all' is your DataFrame
columns_to_convert = ['CV', 'Engine Power', 'Urban Consumption', 'Extra Urban Consumption', 'Combined Consumption', 'CO2 Emissions', 'Fiscal Power']

# Convert columns to float, using pd.to_numeric to handle non-numeric values
for col in columns_to_convert:
    data_all[col] = pd.to_numeric(data_all[col], errors='coerce')

# Check the data_all after conversion
print(data_all[columns_to_convert].dtypes)


(186030, 36)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 186030 entries, 0 to 40051
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Brand                    186030 non-null  object 
 1   Model                    186030 non-null  object 
 2   Type Mines               118002 non-null  object 
 3   CNIT                     139943 non-null  object 
 4   Fuel Type                167469 non-null  object 
 5   CV                       14178 non-null   object 
 6   Engine Power             185838 non-null  object 
 7   Transmission             186030 non-null  object 
 8   Urban Consumption        185920 non-null  object 
 9   Extra Urban Consumption  185920 non-null  object 
 10  Combined Consumption     185923 non-null  object 
 11  CO2 Emissions            185923 non-null  object 
 12  YEAR                     186030 non-null  int64  
 13  Unnamed                  13701 non-null   objec

In [69]:
# Filling empty entries with nans 

# Assuming 'data_all' is your DataFrame
# Fill empty entries (None or empty strings) with NaN
data_all.replace('', np.nan, inplace=True)  # Replace empty strings with NaN
data_all.replace({None: np.nan}, inplace=True)  # Replace None with NaN (if any)

# Check the DataFrame to see the changes
print(data_all.isnull().sum())  # Show the count of NaNs in each column


Brand                           0
Model                           0
Type Mines                  68028
CNIT                        46087
Fuel Type                   18561
CV                         171895
Engine Power                  409
Transmission                    0
Urban Consumption           23811
Extra Urban Consumption     22024
Combined Consumption        23065
CO2 Emissions                 115
YEAR                            0
Unnamed                    172329
Energy Label                92184
Fiscal Power                92075
Registration               142824
Puissance max              185892
Unnamed.2                  186027
Unnamed.3                  186026
CINT                       177771
Bonus/Malus                125859
puiss_admin_98             108162
V9 Field                   108266
Model.2                    145978
dscom                      145978
hybride                    145978
co_typ_1                   146125
hc                         177673
nox           

In [75]:
# Saving the filtered data in one df in the data folder 

# Define the file path for saving the CSV
data_path

# Define the file path for saving the CSV
output_file_path = data_path + '/data_all_french.csv'

print(data_path)

# Save the combined DataFrame to a CSV file
data_all.to_csv(output_file_path, index=False)

print(f"Combined DataFrame saved to {output_file_path}")

/Users/tillo/DataScienceRepo/Project_DS_CO2/data
Combined DataFrame saved to /Users/tillo/DataScienceRepo/Project_DS_CO2/data/data_all_french.csv
